In [2]:
import numpy as np
import pandas as pd
import sys
import time

## The raw data files have been concatenated into a single csv file
### Load it:

In [3]:
path = 'data/processed/LoanStats-combined-z.csv'
t1 = time.time()
df = pd.read_csv(path)
t2 = time.time()
print "{} bytes loaded in {:0.3f} secs".format(sys.getsizeof(df),t2-t1)
print df.shape


/Users/stephane/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


1839704399 bytes loaded in 20.421 secs
(887382, 114)


In [4]:
print list(df.columns)

['lc_id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_il_6m', 'open_il_12m', 'open

In [5]:
df.head(1)

,lc_id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,lc_zip3,zil_coef,zil_interc
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,860,NaN,NaN


## Try to find categorical features, & verify data types

Categorical features are most likely string types with a handful of unique values. Try to identify those here:

In [6]:
num_unique_items = [ df[x].unique().shape[0] for x in df]
print 'numvalues, index, type, name'
for x in sorted(zip(num_unique_items,range(len(df.columns)),[type(df[x][0]) for x in df.columns],df.columns)):
    print x

numvalues, index, type, name
(1, 51, <type 'numpy.int64'>, 'policy_code')
(2, 5, <type 'str'>, 'term')
(2, 17, <type 'str'>, 'pymnt_plan')
(2, 35, <type 'str'>, 'initial_list_status')
(2, 52, <type 'str'>, 'application_type')
(3, 14, <type 'str'>, 'verification_status')
(4, 55, <type 'float'>, 'verification_status_joint')
(6, 12, <type 'str'>, 'home_ownership')
(6, 99, <type 'numpy.float64'>, 'num_tl_120dpd_2m')
(6, 100, <type 'numpy.float64'>, 'num_tl_30dpd')
(7, 8, <type 'str'>, 'grade')
(9, 16, <type 'str'>, 'loan_status')
(9, 56, <type 'numpy.float64'>, 'acc_now_delinq')
(11, 78, <type 'numpy.float64'>, 'chargeoff_within_12_mths')
(12, 11, <type 'str'>, 'emp_length')
(13, 49, <type 'numpy.float64'>, 'collections_12_mths_ex_med')
(13, 61, <type 'numpy.float64'>, 'open_il_12m')
(13, 105, <type 'numpy.float64'>, 'pub_rec_bankruptcies')
(14, 20, <type 'str'>, 'purpose')
(14, 59, <type 'numpy.float64'>, 'open_acc_6m')
(18, 62, <type 'numpy.float64'>, 'open_il_24m')
(19, 66, <type 'numpy

## Categoricals probably have fewer unique values, look for those (they will also have at least 2 different values, otherwise they're not a variable). Also list the actual values.

In [7]:
nuis = [ df.columns[i] for i, x in enumerate(num_unique_items) if (x < 20) & (x > 1)]
for x in nuis:
    print "'{}'".format(x)
    print df[x].value_counts()
    print

'term'
 36 months    621127
 60 months    266255
Name: term, dtype: int64

'grade'
B    254535
C    245860
A    148203
D    139543
E     70705
F     23047
G      5489
Name: grade, dtype: int64

'emp_length'
10+ years    291570
2 years       78870
< 1 year      70606
3 years       70026
1 year        57095
5 years       55704
4 years       52529
n/a           44825
7 years       44595
8 years       43955
6 years       42950
9 years       34657
Name: emp_length, dtype: int64

'home_ownership'
MORTGAGE    443558
RENT        356118
OWN          87471
OTHER          182
NONE            50
ANY              3
Name: home_ownership, dtype: int64

'verification_status'
Source Verified    329559
Verified           291071
Not Verified       266752
Name: verification_status, dtype: int64

'loan_status'
Current                                                573404
Fully Paid                                             238357
Charged Off                                             53599
Late (31-120 

## There are several obvious categoricals based on the above list

### These first two I will treat as ordered, since it seems there is a logical ordering of their values:
- `'term'` - the length of the loan, treating it as an ordinal
- `'emp_length'` - can also be treated as an ordinal, I think

### Lending Club loan grade:
- `'grade'` - this I'll eventually treat as a response, based on previous analysis. It's already a ranking of the probability that the loan will be paid back.

### And the categoricals:
- ```
'home_ownership', 'verification_status', 'pymnt_plan','purpose','initial_list_status','application_type','verification_status_joint'
                    ```
                    
### The remaining variables in the short list all have numeric values, so I'm going to leave them that way.

## Process the unordered categoricals:

In [8]:
cat_feature_names = ['home_ownership', 'verification_status', 
                     'pymnt_plan','purpose','initial_list_status',
                    'application_type','verification_status_joint']

df_catf = pd.get_dummies(df[cat_feature_names])
df=pd.concat([df,df_catf],axis = 1)
print df.shape

(887382, 146)


## Process the ordered categoricals:

In [9]:
ord_feature_names = ['term', 'emp_length']
ord_feature_numnames = [x + '_num' for x in ord_feature_names]
ord_feature_map = [
    ['term', {' 36 months' :0, ' 60 months' : 1}], 
    ['emp_length', {'< 1 year': .5, '1 year' : 1, '2 years' : 2, '3 years' : 3, '4 years':4, '5 years':5, 
                   '6 years':6, '7 years':7, '8 years':8, '9 years':9, '10+ years':10,
                   'n/a': 0 }]
]

In [10]:
#minidf = df.head(30) # for testing dummies & categorical conversions
for x in ord_feature_map:
    df[x[0]+'_num']=df[x[0]].map(x[1])
#print df.head()
print df[ord_feature_names].head()
print df[ord_feature_numnames].head()

         term emp_length
0   36 months  10+ years
1   60 months   < 1 year
2   36 months  10+ years
3   36 months  10+ years
4   60 months     1 year
   term_num  emp_length_num
0         0            10.0
1         1             0.5
2         0            10.0
3         0            10.0
4         1             1.0


In [11]:
print df.shape

(887382, 148)


## Convert the response variable to a binary representation

`'loan_status'` Is what I eventually want to predict. It has many possible values, but the majority of them can be simplified to either a 'successful' loan or a 'default' loan

In [12]:
df.loan_status.value_counts()

Current                                                573404
Fully Paid                                             238357
Charged Off                                             53599
Late (31-120 days)                                      11339
In Grace Period                                          5346
Late (16-30 days)                                        2297
Does not meet the credit policy. Status:Fully Paid       1988
Does not meet the credit policy. Status:Charged Off       761
Default                                                   291
Name: loan_status, dtype: int64

In [13]:
df['loan_status_num'] = df.loan_status.map(
    {'Charged Off':0, 'Default':0, 'Fully Paid':1, 'Current':1})
tot_good_loans = df.loan_status_num[df.loan_status_num == 1].count()
total_success_rate = float(tot_good_loans)/df.loan_status_num.count()
print "overall success rate for loans({}/{}):  {}% ".format(
    tot_good_loans, df.loan_status_num.count(), total_success_rate)

overall success rate for loans(811761/865651):  0.937746274191% 


## Modify 'grade'

I can use this for comparison and/or verification during parts of the analysis

In [14]:
df.grade.value_counts().sort_index()

A    148203
B    254535
C    245860
D    139543
E     70705
F     23047
G      5489
Name: grade, dtype: int64

In [15]:
df['grade_num'] = df.grade.map({'A':0, 'B':1, 'C':2, 'D':3, 'E':4, 'F':5, 'G':6})
gradenames=sorted(df.grade.unique())
gradenums = np.arange(0,7)


### Save this version of the file

In [16]:
pwd

u'/Users/stephane/classes/ds2/repos/gadat-project'

In [18]:
print list(df.columns)

['lc_id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_il_6m', 'open_il_12m', 'open

In [19]:
path = 'data/processed/LoanStats-combined-z-2.csv'
t1 = time.time()
df.to_csv(path,index = False)
print df.shape
t2 = time.time()
print t2-t1

(887382, 150)
155.400627136
